# Coarse-grained MD Simulation Data Provenance with ``aiida-gromacs``

## Introduction

### Who is this tutorial for?
This tutorial is aimed at molecular dynamics simulators who want to keep track of each step used to build molecular systems and perform simulations of biomolecular systems. 

### Why not just use a script to keep a record of simulation steps?
As simulation protocols vary between practictioners, it is often difficult to ascertain how a simulation was performed to generate the final dynamics of biomolecules

### How can we track complex simulation protocols in a FAIR way?
In this tutorial, we will setup and run a coarse-grained simulation of the active state PTH2R protein via the ``aiida-gromacs`` plugin. We will show how commands are written and saved in realtime.
The provenance of each command will be recorded in realtime and the files inputted and outputted for each command is saved in the aiida database


## Software requirements

* [``martinize2``](https://pypi.org/project/vermouth/) is used to convert from atomistic to coarse-grained structures. 

* A modified [``insane``](https://github.com/Tsjerk/Insane) script is used to build the coarse-grained system, which is available here XXX.

* [GROMACS](https://www.gromacs.org/) is required to perform molecular dynamics simulations.

* [``aiida-gromacs``](https://aiida-gromacs.readthedocs.io/en/latest/user_guide/installation.html#plugin-installation) is used to keep track of all the commands used to setup and perform the simulation. 

Ensure the conda environment that `aiida-gromacs` and AiiDA are installed in is activated for this tutorial. We then need to start the AiiDA database and daemon using the first three steps of our [User guide](https://aiida-gromacs.readthedocs.io/en/latest/user_guide/aiida_sessions.html#start-stop-aiida).

## AiiDA under the hood

We use ``aiida-gromacs`` to track commands run on the CLI, which is a plugin for AiiDA software. Here's a brief description of what's going on under the hood when running ``aiida-gromacs``; 
* AiiDA uses a [PostgreSQL](https://www.postgresql.org) database to store all data produced and the links between input and output files for each command run. Each submitted command is termed a process in AiiDA. 
* Communication between submitted processes are handled with [RabbitMQ](https://www.rabbitmq.com/) and submitted processes are handled with a deamon process that runs in the background. 
* AiiDA has a built-in CLI utility called ``verdi``, which we will use to view the status of the submitted process before moving onto the next step, you do this with the following command:

In [ ]:
! verdi process list -a

A successfully finished process will exit with code [0]. 

## Aquiring and tidying up the receptor protein structure

We are using the PTH2R structure from the orientations of proteins in membranes (OPM) database as our starting point, available [here](https://opm.phar.umich.edu/proteins/7900). This structure is correctly orientated to fit around a membrane and uses the [7F16](https://www.rcsb.org/structure/7F16) PDB deposited structure.

1. First we will download the protein using `curl` and track the aquisition of the file using the `genericMD` command in `aiida-gromacs`: 


In [ ]:
%cd PTH2R_coarse-grained_files/protein

In [ ]:
! genericMD --code bash@localhost \
--command "curl https://opm-assets.storage.googleapis.com/pdb/7f16.pdb -o 7f16_opm.pdb" \
--outputs 7f16_opm.pdb

2. PTH2R is a receptor for the parathyroid hormone and the downloaded structure contains the coupled G-protein along with other bound molecules. We will keep only the receptor using the `sed` command by removing lines that do not correspond to the receptor:


In [ ]:
! genericMD --code bash@localhost \
--command "sed -i -e '1,761d;3834,13707d' 7f16-opm.pdb" \
--inputs 7f16_opm.pdb --outputs 7f16_opm.pdb

## Building a coarse-grained system from an atomic structure

Now that we have the correct starting structure, we move onto coarse-graininge.

3. We use Martinize2 to coarse-grain the atomistic structure and produce a GROMACS topology file

In [ ]:
%cp 7f16_opm.pdb ../martinize
%cd ../martinize


In [ ]:
! genericMD --code martinize2@localhost --command "-f 7f16_opm.pdb -o PTH2R_opm.top -x PTH2R_opm.cg.pdb -ff martini3001 -nt -elastic -p backbone -maxwarn 1 -mutate HSD:HIS -mutate HSP:HIH -ignh -cys auto -scfix" \
--inputs 7f16_opm.pdb \
--outputs PTH2R_opm.top --outputs PTH2R_opm.cg.pdb --outputs molecule_0.itp

4. Next, we use our custom insane python script to embed the protein into a lipid bilayer and solvate the system. Our insane script is modified from [the Melo lab](https://github.com/MeloLab/PhosphoinositideParameters/blob/main/martini3/insane.py), it has been updated to python3 and contains additional parameters for the GM3 carbohydrate. 

In [ ]:
! cp PTH2R_opm.cg.pdb PTH2R_opm.top molecule_0.itp ../insane
%cd ../insane

In [ ]:

! genericMD --code python@localhost --command "insane_custom.py -f PTH2R_opm.cg.pdb -o solvated.gro -p solvated.top -pbc rectangular -box 18,18,17 -u POPC:25 -u DOPC:25 -u POPE:8 -u DOPE:7 -u CHOL:25 -u DPG3:10 -l POPC:5 -l DOPC:5 -l POPE:20 -l DOPE:20 -l CHOL:25 -l POPS:8 -l DOPS:7 -l POP2:10 -sol W" \
--inputs insane_custom.py --inputs PTH2R_opm.cg.pdb \
--outputs solvated.gro --outputs solvated.top

5. Once the topology file is created, we need to include all the itp files that contain the force field parameters used to describe interactions between beads. We use the `sed` command to edit the `system.top` file directly on the command-line and we submit this commad via `aiida-gromacs` as with the previous commands

In [ ]:
sed_command1='sed -i -e "1 s/^/#include \\"toppar\/martini_v3.0.0.itp\\"\\n#include \\"toppar\/martini_v3.0.0_ions_v1.itp\\"\\n#include \\"toppar\/martini_v3.0.0_solvents_v1.itp\\"\\n#include \\"toppar\/martini_v3.0.0_phospholipids_v1.itp\\"\\n#include \\"martini_v3.0_sterols_v1.0.itp\\"\\n#include \\"POP2.itp\\"\\n#include \\"molecule_0.itp\\"\\n#include \\"gm3_final.itp\\"\\n/" '\
'-e "s/Protein/molecule_0/" '\
'-e "s/#include \\"martini.itp\\"/\\n/" solvated.top'
! genericMD --code bash@localhost --command '{sed_command1}' --inputs solvated.top --outputs solvated.top

6. We also need to edit the `molecule_0.itp` file generated from the Martinize2 step to include positional restraints on the coarse-grained beads

In [ ]:
sed_command2='sed -i -e "s/1000 1000 1000/POSRES_FC    POSRES_FC    POSRES_FC/g" '\
'-e "s/#ifdef POSRES/#ifdef POSRES\\n#ifndef POSRES_FC\\n#define POSRES_FC 1000.00\\n#endif/" '\
'molecule_0.itp'
! genericMD --code bash@localhost --command '{sed_command2}' --inputs molecule_0.itp --outputs molecule_0.itp

7. Ions need to be added to the system and we can construct the GROMACS `.tpr` binary file that contains the system configuration, topology and input parameters for the next step. We use the `gmx_grompp` command (note the underscore), which is wrapper command to run `gmx` via `aiida-gromacs`

In [ ]:
! cp molecule_0.itp solvated.gro solvated.top ../gromacs
%cd ../gromacs

In [ ]:
! gmx_grompp -f ions.mdp -c solvated.gro -p solvated.top -o ions.tpr

8. The `gmx_genion` command is then used to add the ions to reach a particular salt concentration and neutralise the system. As the `genion` command requires interactive user inputs, we can provide these in as an additional text file via the `--instructions` argument. Each interactive response can be provided on a new line in the input text file. In this example, we replace solvent `W` with ions

In [ ]:
! gmx_genion -s ions.tpr -o solvated_ions.gro -p solvated.top -pname NA -nname CL -conc 0.15 -neutral true --instructions inputs_genion.txt

9. Lastly, we will use a `gmx_make_ndx` to create new index groups for the membrane and solute consituents

In [ ]:
! gmx_make_ndx -f solvated_ions.gro -o index.ndx --instructions inputs_index.txt

## Saving and viewing all the steps used to build the system

We have built our starting configuration of an embedded protein in a lipid bilayer, hurray!

To view all the currently run commands in each process, the input files used in each command and the output files produced from each command, we can use `print_provenance` to do this:

In [ ]:
! print_provenance

We can also visualise the provenance graph of these processes, which shows how inputs and outputs of each process are connected to other processes. To save the provenance graph of all finished processes, replace the primary key value <PK> in the command below with that of the most recently run process.

In [ ]:
! verdi node graph generate <PK>

TODO: Add image of a section of the provenance graph as an example

At the end of a project, the AiiDA database can be saved as an AiiDA archive file (sqlite/zip format) for long term storage and to share your data and provenance with others. This archive file contains all the input and output files for each process, as well as how they are connected.

In [ ]:
! verdi archive create --all archive.aiida

We hope to share further tutorials on loading, querying and displaying data from AiiDA archives. Watch this space!

## Preparing system for production simulation